In [2]:
#!/usr/bin/env python
from __future__ import print_function
import os
import time
import yaml
import pprint
import random
import pickle
import shutil
import inspect
import argparse
from collections import OrderedDict, defaultdict

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
# from tensorboardX import SummaryWriter
from torch.optim.lr_scheduler import MultiStepLR
import apex

from utils import count_params, import_class

from torch.utils.data import RandomSampler

In [2]:
print(os.cpu_count())

12


In [6]:
import torch
import numpy as np

x=torch.rand(2,3,2,5,4)
a=torch.rand(3,4,4)
print("x: ")
print(x)
print('A: ')
print(a)
x=torch.einsum('nkctv,kvw->nctw',x,a)
print("Einsum: ")
print(x)

x: 
tensor([[[[[8.6850e-01, 3.5980e-01, 4.1007e-01, 3.3197e-01],
           [9.8913e-01, 7.0587e-01, 3.5317e-01, 2.7693e-01],
           [4.7916e-01, 3.4757e-01, 3.0264e-01, 7.5698e-01],
           [3.9869e-01, 4.4909e-01, 8.6481e-01, 8.0092e-01],
           [6.1321e-01, 9.6844e-01, 1.3324e-01, 3.3555e-01]],

          [[8.9902e-01, 3.8167e-01, 9.0704e-01, 6.2656e-01],
           [7.3679e-01, 7.8089e-01, 8.3425e-01, 5.5853e-01],
           [8.8177e-01, 7.3485e-02, 5.7952e-01, 9.9550e-01],
           [3.3214e-01, 3.0591e-01, 8.9580e-01, 4.0680e-01],
           [9.9721e-01, 1.9325e-01, 3.0268e-01, 3.6518e-01]]],


         [[[6.2763e-01, 4.3258e-01, 4.8501e-02, 6.8368e-02],
           [8.1530e-01, 8.1021e-01, 3.8821e-01, 9.6113e-01],
           [6.1757e-01, 3.4493e-01, 3.7429e-01, 5.2310e-02],
           [8.4370e-02, 3.7160e-01, 2.9437e-01, 8.2659e-01],
           [3.9602e-01, 3.4873e-01, 2.5265e-01, 7.5918e-01]],

          [[4.1732e-01, 1.5487e-01, 1.3140e-01, 4.9684e-01],
           [

In [3]:
import torch
import numpy as np

x=torch.rand(2, 3, 4)

a=torch.rand(2, 3, 4, 5)
print('A: ')
print(a)
# print('A.reshape: ')
# print(a.reshape(5,3,4))

print("x: ")
print(x)

e=torch.einsum('bkv,nkvw->nw',x,a)
print("Einsum: ")
print(e)
print(f'xshape = {e.shape}')

A: 
tensor([[[[0.7153, 0.4327, 0.5727, 0.0226, 0.3009],
          [0.0865, 0.9918, 0.7755, 0.9348, 0.3814],
          [0.0602, 0.5021, 0.1960, 0.0329, 0.7702],
          [0.7422, 0.2233, 0.4768, 0.4998, 0.5712]],

         [[0.5226, 0.9427, 0.3542, 0.5468, 0.2033],
          [0.7005, 0.5211, 0.1534, 0.9765, 0.4844],
          [0.9033, 0.8965, 0.7308, 0.6766, 0.7726],
          [0.4787, 0.5511, 0.9350, 0.8405, 0.4020]],

         [[0.1057, 0.7628, 0.9024, 0.7178, 0.9714],
          [0.3199, 0.2599, 0.4803, 0.6697, 0.9276],
          [0.4631, 0.5747, 0.2404, 0.7714, 0.5269],
          [0.9600, 0.5270, 0.5665, 0.7310, 0.4193]]],


        [[[0.6133, 0.2456, 0.7161, 0.4619, 0.8046],
          [0.5446, 0.4426, 0.2918, 0.0662, 0.0981],
          [0.3425, 0.6403, 0.6798, 0.6080, 0.8461],
          [0.2414, 0.0862, 0.1804, 0.4354, 0.8651]],

         [[0.0376, 0.3780, 0.9747, 0.9933, 0.9810],
          [0.6943, 0.2406, 0.3773, 0.7926, 0.1308],
          [0.8356, 0.3050, 0.7834, 0.7672, 0.3001]

In [5]:
for n in range(2):
    for w in range(5):
        total=0
        for k in range (3):
            for v in range(4):
                for b in range(2):
                    total+=x[b,k,v]*a[n,k,v,w]
        print(total)


tensor(5.5368)
tensor(7.1013)
tensor(6.5095)
tensor(7.7100)
tensor(6.9732)
tensor(5.1312)
tensor(4.7079)
tensor(6.2078)
tensor(5.5375)
tensor(6.4076)


In [31]:
a.reshape(5, 3, 4)

tensor([[[0.8628, 0.6923, 0.4384, 0.4074],
         [0.1809, 0.7386, 0.4199, 0.8970],
         [0.4374, 0.2622, 0.4879, 0.3475]],

        [[0.5463, 0.5022, 0.2446, 0.1330],
         [0.0874, 0.0602, 0.5923, 0.4944],
         [0.5609, 0.3987, 0.5275, 0.2604]],

        [[0.5661, 0.8224, 0.5832, 0.9938],
         [0.1841, 0.7262, 0.8141, 0.2486],
         [0.8873, 0.3361, 0.3435, 0.9783]],

        [[0.6308, 0.4075, 0.7316, 0.3681],
         [0.6317, 0.3352, 0.2272, 0.1452],
         [0.3326, 0.2005, 0.4277, 0.6551]],

        [[0.9782, 0.2696, 0.4653, 0.8867],
         [0.4663, 0.0037, 0.8986, 0.1719],
         [0.7821, 0.0132, 0.1104, 0.9301]]])

In [7]:
a=[1,2,3,4]
b=[2,2,2,2]

c=[a[e] for e in range(3) for i in range(i)]
print(c)

UnboundLocalError: local variable 'i' referenced before assignment

In [45]:
import torch

a=torch.rand(1,1,1200,25,25)
print(a)

tensor([[[[[0.2138, 0.2214, 0.0647,  ..., 0.0014, 0.9469, 0.3007],
           [0.9779, 0.7906, 0.9681,  ..., 0.8033, 0.3671, 0.6881],
           [0.0525, 0.5650, 0.4648,  ..., 0.8935, 0.1577, 0.9718],
           ...,
           [0.5324, 0.8295, 0.1725,  ..., 0.7387, 0.3392, 0.5777],
           [0.9179, 0.3528, 0.4956,  ..., 0.0650, 0.3672, 0.4534],
           [0.2192, 0.5900, 0.1589,  ..., 0.6699, 0.1947, 0.4691]],

          [[0.7521, 0.9064, 0.7631,  ..., 0.1040, 0.8198, 0.2050],
           [0.2142, 0.8551, 0.5893,  ..., 0.4807, 0.8102, 0.3321],
           [0.8332, 0.2443, 0.2138,  ..., 0.9781, 0.4639, 0.1250],
           ...,
           [0.2410, 0.1478, 0.9523,  ..., 0.5249, 0.7073, 0.1396],
           [0.2485, 0.4021, 0.9239,  ..., 0.2172, 0.8970, 0.9158],
           [0.7298, 0.2467, 0.2520,  ..., 0.6545, 0.1153, 0.5954]],

          [[0.3206, 0.1442, 0.1526,  ..., 0.1537, 0.1453, 0.8373],
           [0.2949, 0.4639, 0.4748,  ..., 0.9872, 0.1668, 0.7656],
           [0.9023, 0.1454

In [47]:
b=a
b=torch.nn.Conv3d(1,1,(300,3,3),padding=(0,1,1),stride=(300,1,1))(b)
print(b.shape)

torch.Size([1, 1, 4, 25, 25])


In [63]:
ca=torch.rand(5,2,3,4)
a=torch.rand(1,1,5,4,4)
print("ca: ",ca)
print("a: ",a)


ca:  tensor([[[[0.4023, 0.2946, 0.0197, 0.4110],
          [0.9954, 0.9065, 0.0214, 0.1849],
          [0.8778, 0.0591, 0.5950, 0.6955]],

         [[0.1523, 0.5570, 0.6428, 0.0310],
          [0.2899, 0.9677, 0.5205, 0.9125],
          [0.6758, 0.4344, 0.5289, 0.1656]]],


        [[[0.9500, 0.3794, 0.8305, 0.6778],
          [0.8053, 0.2734, 0.0238, 0.3144],
          [0.1611, 0.0756, 0.0558, 0.7051]],

         [[0.7464, 0.8235, 0.0959, 0.4941],
          [0.1438, 0.4464, 0.7240, 0.9332],
          [0.1313, 0.7375, 0.3688, 0.3534]]],


        [[[0.8220, 0.7803, 0.3809, 0.6885],
          [0.5230, 0.0737, 0.1200, 0.8044],
          [0.3120, 0.8600, 0.3777, 0.6560]],

         [[0.9526, 0.4164, 0.8973, 0.9840],
          [0.9437, 0.0839, 0.2370, 0.0892],
          [0.7993, 0.1283, 0.4784, 0.1022]]],


        [[[0.5640, 0.0874, 0.1597, 0.9912],
          [0.7872, 0.9474, 0.0570, 0.0029],
          [0.2157, 0.5690, 0.8554, 0.8887]],

         [[0.4182, 0.7160, 0.0352, 0.8499],
       

In [64]:
print(torch.stack([torch.einsum("vu,ctu->ctv",a[0,0,i], ca[i]) for i in range(5)]))

tensor([[[[0.6545, 0.1614, 1.0337, 0.4873],
          [0.9859, 0.4649, 1.9324, 0.5604],
          [1.3765, 0.2845, 1.5606, 1.3114]],

         [[0.4481, 0.4718, 0.7433, 0.8201],
          [1.2547, 0.6208, 2.0810, 1.5206],
          [0.8567, 0.4119, 1.2217, 0.8785]]],


        [[[1.1093, 1.4092, 0.7714, 0.9533],
          [0.5969, 0.8819, 0.1613, 0.4768],
          [0.6333, 0.5680, 0.1680, 0.5811]],

         [[1.1240, 1.3489, 0.3137, 0.6713],
          [1.1754, 1.0765, 0.6945, 0.9190],
          [0.8783, 0.8743, 0.4147, 0.4519]]],


        [[[1.7765, 1.1592, 1.2445, 0.8861],
          [0.8818, 0.8717, 0.6575, 0.3309],
          [1.3498, 0.6571, 0.8969, 0.9031]],

         [[2.2469, 1.4349, 1.8257, 1.2111],
          [1.1493, 0.9687, 0.9237, 0.2861],
          [1.2716, 0.8542, 1.0720, 0.5120]]],


        [[[1.0487, 0.9660, 1.4228, 0.7891],
          [1.4836, 1.3618, 0.9620, 1.3208],
          [1.1265, 1.6156, 1.5563, 1.6335]],

         [[1.3574, 1.0939, 1.1930, 1.2005],
          [1

In [65]:
# a_reshaped = a.squeeze(0).squeeze(0)
print(a[0,0].shape)
result = torch.einsum("nuv, nctu->nctv", a[0,0], ca)
print(result)

torch.Size([5, 4, 4])
tensor([[[[0.3884, 0.3081, 0.6531, 0.6609],
          [0.7987, 0.6166, 0.8506, 0.9496],
          [1.2767, 0.8881, 1.0718, 1.8043]],

         [[0.7285, 0.8787, 0.3274, 0.7558],
          [0.8953, 1.1740, 1.3793, 1.4483],
          [1.0050, 0.8325, 0.5973, 1.1641]]],


        [[[0.6598, 1.1411, 0.9038, 1.5709],
          [0.4138, 0.8026, 0.2604, 1.0072],
          [0.2661, 0.2566, 0.2161, 0.6946]],

         [[0.7989, 1.1796, 0.4387, 1.4202],
          [0.6014, 0.6308, 0.7664, 1.1648],
          [0.6061, 0.7078, 0.4733, 0.8395]]],


        [[[1.7764, 0.9482, 1.7681, 0.8358],
          [0.6472, 0.7411, 1.2857, 0.4017],
          [1.3892, 0.6312, 1.2733, 0.7017]],

         [[1.9103, 1.2635, 2.6200, 0.8852],
          [1.1006, 0.6595, 1.1829, 0.4033],
          [1.1809, 0.6321, 1.3009, 0.4285]]],


        [[[1.1635, 1.5227, 1.0159, 0.7403],
          [1.5874, 1.2755, 0.9967, 0.6640],
          [1.9034, 1.5671, 1.8430, 1.1999]],

         [[1.3986, 1.6555, 1.4107,

In [14]:
x = torch.tensor([[[[2,3],[4,5]],[[6,7],[8,0]]]])
y = torch.tensor([[[[2,3],[4,5]],[[6,7],[8,9]]]])
n, t, v, u = x.shape
n, t, c, u = y.shape
m = torch.zeros(n,t,c,v)
for i in range(n):
    for f in range(t):
        for j in range(v):
            for ch in range (c):
                total = 0
                for uv in range (u):
                    total += x[i,f,j,uv]*y[i,f,ch,uv]
                m[i,f,ch,j] = total
print(x.shape)
print(torch.einsum('ntvu,ntcu->ntcv',x,y))

print(m)

torch.Size([1, 2, 2, 2])
tensor([[[[ 13,  23],
          [ 23,  41]],

         [[ 85,  48],
          [111,  64]]]])
tensor([[[[ 13.,  23.],
          [ 23.,  41.]],

         [[ 85.,  48.],
          [111.,  64.]]]])
